# Загрузка Pandas и очистка данных

In [436]:
import pandas as pd
from collections import Counter
import re
import datetime
from datetime import datetime, timedelta
import numpy as np

In [437]:
df = pd.read_csv('main_task_new.csv') #Загружаем датасет

## Предобработка

In [438]:

# Найдём даты:
pattern = re.compile('\d\d/\d\d/\d\d\d\d')

# Преобразование в формат datetime:
def to_datetime(time):
    datetime_list = []
    for date in time:
        dt = datetime.strptime(date, '%m/%d/%Y')
        datetime_list.append(dt)
    return datetime_list

# Cоздадим словарь для замены
price_dict = {'$' : 1,
                '$$ - $$$' : 2,
                '$$$$' : 3,
                }
# Заменим на числовые значения
df['Price Range'] = df['Price Range'].replace(to_replace = price_dict)
df['Price Range'].fillna(df['Price Range'].median(), inplace = True)


In [439]:
# Посчитаем количество уникальных значений в столбце Restaurant_id
# Ищем сети ресторанов
restaurant_with_freqs = list(df['Restaurant_id'].value_counts())
len(restaurant_with_freqs)

11909

In [440]:
# Оставим только рестораны, описывающие 95% примеров
top_restaurant_count = int(np.percentile(restaurant_with_freqs, 95))
top_restaurant_count

11

In [441]:
# Оставим только этот топ, а остальные рестораны назовём other
all_restaurants = df['Restaurant_id'].value_counts().index
top_restaurants = list(all_restaurants)[:top_restaurant_count]
restaurants_to_throw_away = list(set(all_restaurants) - set(top_restaurants))
df.loc[df['Restaurant_id'].isin(restaurants_to_throw_away), 
             'restaurant'] = 'other'

In [442]:
# Заполним пропуски средним значением
df['Number of Reviews'].fillna(0, inplace = True)

In [443]:
# Обработка столбца "Cuisine Style":
# Посчитаем, сколько типов кухонь представлено в каждом ресторане. В случае
# пропуска, будем считать, что в ресторане представлен один тип кухни

df['Cuisine Style'].fillna('1', inplace = True)


In [444]:
# Уберём лишние скобки и пробелы:
df['Cuisine Style']=df['Cuisine Style'].apply(lambda s: str(s).replace('[',''))
df['Cuisine Style']=df['Cuisine Style'].apply(lambda s: str(s).replace(']',''))
df['Cuisine Style']=df['Cuisine Style'].apply(lambda s: str(s).replace(' ',''))

In [445]:
# Предствим кухни в виде списка:
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda s: str(s).split(','))

In [446]:
# Посчитаем количество кухонь для каждого ресторана:
df['cuisine_count'] = df['Cuisine Style'].apply(lambda s: len(s))

In [447]:
# Найдём даты отзывов:
df['date'] = df['Reviews'].apply(lambda s:pattern.findall(s))

In [448]:
# Преобразуем в формат datetime:
df['date'] = df['date'].apply(to_datetime)


In [449]:
# Посчитаем интервал между датами публикаций отзывов, размещённых на сайте:
df['date_1'] = df['date'].apply(lambda s: len(s)) 
df['date_interval'] = df[df.date_1 > 1]['date'].apply(lambda s: abs(s[0]-s[1]))


In [450]:
# заменим пропуски средним значением:
df['date_interval'].fillna(df['date_interval'].median(), inplace = True)

In [451]:
# преобразуем в float
df['date_interval'] = df['date_interval'].apply( lambda s: str(s).split(' '))
df['date_interval'] = df['date_interval'].apply(lambda s: float(s[0]))

In [452]:
# дата первого отзыва
df['date1'] = df[df['date_1']>0]['date'].apply(lambda s: s[0]) 

In [453]:
# дата второго отзыва
df['date2'] = df[df['date_1']>1]['date'].apply(lambda s: s[1])

In [454]:
# дата последнего отзыва
max_date = max(df.date.max())
max_date


datetime.datetime(2018, 2, 26, 0, 0)

In [426]:
# проверим, насколько давно были оставлены отзывы, относительно последнего

df['date1'] = df[df['date_1']>0]['date1'].apply(lambda s: max_date - s)
df['date2'] = df[df['date_1']>1]['date2'].apply(lambda s: max_date - s) 

In [427]:
# заменим пропуски средним значением:
df['date1'].fillna(df['date1'].median(), inplace = True)
df['date2'].fillna(df['date2'].median(), inplace = True)

In [428]:
# преобразуем в float
df['date1'] = df['date1'].apply( lambda s: str(s).split(' '))
df['date1'] = df['date1'].apply(lambda s: float(s[0]))

df['date2'] = df['date2'].apply( lambda s: str(s).split(' '))
df['date2'] = df['date2'].apply(lambda s: float(s[0]))

In [429]:
# Переведём категориальные переменные в числовые
df = pd.get_dummies(df, columns = ['City', 'restaurant'],
                    dummy_na = True)

In [430]:
# Уберём неподходящие столбцы:
df.drop(['ID_TA'], inplace = True, axis = 1)
df.drop(['URL_TA'], inplace = True, axis = 1)
df.drop(['Cuisine Style'], inplace = True, axis = 1)
df.drop(['Reviews'], inplace = True, axis = 1)
df.drop(['Restaurant_id'], inplace = True, axis =1)
df.drop(['date'], inplace = True, axis = 1)
df.drop(['date_1'], inplace = True, axis = 1)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [431]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги
# ресторанов)
X = df.drop(['Rating'], axis = 1)
y = df['Rating']

In [28]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [432]:
# Наборы данных с меткой "train" будут использоваться для обучения модели,
# "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [30]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания
# и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [433]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой
# выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [434]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим
# насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение
# предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2090885


In [284]:
df

,Ranking,Rating,Number of Reviews,cuisine_count,date_interval,date1,date2,restaurant_other,Price Range_$,Price Range_$$ - $$$,...,City_Munich,City_Oporto,City_Oslo,City_Paris,City_Prague,City_Rome,City_Stockholm,City_Vienna,City_Warsaw,City_Zurich
0,5570.0,3.5,194.0,3,41.0,57.0,98.0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
1,1537.0,4.0,10.0,1,382.0,235.0,617.0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,353.0,4.5,688.0,7,2.0,49.0,51.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3458.0,5.0,3.0,1,67.0,167.0,230.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,621.0,4.0,84.0,3,272.0,100.0,372.0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,500.0,4.5,79.0,4,34.0,72.0,106.0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
39996,6341.0,3.5,542.0,5,9.0,67.0,76.0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
39997,1652.0,4.5,4.0,2,3127.0,480.0,3607.0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
39998,641.0,4.0,70.0,5,23.0,230.0,253.0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
